### Ingestion del archivo "country.json"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

#### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
countries_schema = "countryId INT, countryIsoCode STRING, countryName STRING"

In [0]:
#Agregamos v_file_date, ya que ahora en nuestro Data Lake, tenemos carpetas con la fecha
#Seccion carga incormental, para esta carga por ser pocos datos, sera una carga completa
countries_df = spark.read \
               .schema(countries_schema) \
               .json(f"{bronze_folder_path}/{v_file_date}/country.json")

### Paso 2 - Eliminar columnas no deseadas del DataFrame

In [0]:
countries_dropped_df = countries_df.drop("countryIsoCode")

In [0]:
countries_dropped_df = countries_df.drop(countries_df["countryIsoCode"])

In [0]:
from pyspark.sql.functions import col

In [0]:
countries_dropped_df = countries_df.drop(col("countryIsoCode"))

### Paso 3 - Cambiar el nombre de las columnas y añadir "ingestion_date" y "environment"

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
countries_final_df = add_ingestion_date(countries_dropped_df) \
                     .withColumn("environment", lit(v_environment)) \
                     .withColumn("file_date", lit(v_file_date)) \
                     .withColumnRenamed("countryId", "country_id") \
                     .withColumnRenamed("countryName", "country_name")
                     

### Paso 4 - Escribir la salida en un formato "Parquet"

In [0]:
# countries_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/countries")

# Crear una tabla en base al data frame countries_final_df
# Creara la tabla countries en a DB que se llame movie_silver
# Modulo: Spark SQL - Database / Table / View: seccion: Crear Tabla - Capa Silver
# countries_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.countries")

# Modulo: Delta Lake
countries_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.countries")

In [0]:
%sql
SELECT * FROM movie_silver.countries;

In [0]:
#display(spark.read.parquet(f"{silver_folder_path}/countries"))

In [0]:
dbutils.notebook.exit("Success")